In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install ultralytics
%pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 10.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [ ]:
SEED = 42
BATCH_SIZE = 16
MODEL = "Yolov8_v8"
data_path = '/content/drive/MyDrive/Colab Notebooks/ATL/합성데이터 기반 객체 탐지 AI 경진대회'

In [ ]:
#model = YOLO(f"{data_path}/Yolov8_v3/v3/train/weights/last.pt")
model = YOLO("yolov8x") # yolov8n, yolov8s, yolov8m, yolov8l, yolov8x 등이 있음
results = model.train(
    data= data_path + "/data/yolocustom.yaml",
    imgsz=640,
    epochs=50,
    batch=BATCH_SIZE,
    patience=5,
    workers=16,
    device=0,
    exist_ok=True,
    project=f"{data_path}/{MODEL}",
    name="train",
    seed=SEED,
    pretrained=False,
    resume=False,

    optimizer="AdamW",
    lr0=0.0005,
    mixup=0.3,
    cos_lr=True,
    flipud=0.0,
    fliplr=0.0,

    augment=False,
    val=True,
    cache=True
    )

100%|██████████| 131M/131M [00:07<00:00, 17.3MB/s]
Ultralytics YOLOv8.0.116 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/drive/MyDrive/Colab Notebooks/ATL/합성데이터 기반 객체 탐지 AI 경진대회/data/yolocustom.yaml, epochs=50, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=16, project=/content/drive/MyDrive/Colab Notebooks/ATL/합성데이터 기반 객체 탐지 AI 경진대회/Yolov8_v8, name=train, exist_ok=True, pretrained=False, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_la

In [ ]:
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [ ]:
model = YOLO(f"{data_path}/{MODEL}/train/weights/best.pt")
test_image_paths = glob(data_path + "/data/yolo_v8/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(640, 640), iou=0.2, conf=0.3, save_conf=True, save=False, save_txt=True, project=f"{data_path}/{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 1 == 0:
        clear_output(wait=True)

213it [14:44,  4.15s/it]


In [ ]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = (x - width / 2) * image_width
    x_max = (x + width / 2) * image_width
    y_min = (y - height / 2) * image_height
    y_max = (y + height / 2) * image_height

    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [ ]:
infer_txts = glob(f"{data_path}/{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"{data_path}/data/test/{base_file_name}.png").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"{data_path}/results/{MODEL}.csv", index=False)

100%|██████████| 3399/3399 [1:14:44<00:00,  1.32s/it]
